In [8]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [27]:
def auction_bids(game_info, current_info, last_auction_result, player_information):
    # The bid is calculated as a fraction of the total number of items, 
    # with a small adjustment to avoid rounding issues.
    print(last_auction_result)
    num_robots = game_info['Number_Robots']
    if last_auction_result['winner'] is None:
        player_information['total_points_remaining'] = num_robots + 1.0
        player_information['Robots'] = [1.0] * game_info['Number_Robots']
        player_information['VP'] = {"average": None, "std": None}
        
    else:
        player_information["total_points_remaining"] -=  last_auction_result['total']
        if last_auction_result['winner'].startswith("Robot"):
            winning_robot = int(last_auction_result['winner'].split("_")[1])
            player_information['Robots'][winning_robot - 1] -= last_auction_result['winning_bid']
            
            other_robots_wasted = last_auction_result['total'] - last_auction_result['winning_bid'] - player_information['last_bid']
            min_bid = last_auction_result['min']
            other_robot_indexes = [(i,player_information['Robots'][i]) for i in range(num_robots) if i != winning_robot - 1]
            other_robot_indexes.sort(key = lambda x: x[1], reverse=True)
            other_robot_wasted = [min_bid] + [(other_robots_wasted - min_bid) / (num_robots - 2)] * (num_robots - 2)
            for (index, _), wasted in zip(other_robot_indexes, other_robot_wasted):
                player_information['Robots'][index] -= wasted
        player_information['VP']["average"] = last_auction_result
    print(player_information)
            
                
        
        
    
    bid_value = 1.0 / game_info['Total_Number_Items']  # Rounding Error Fix
    player_information['last_bid'] = bid_value

    return bid_value

In [28]:
import random



def simulate_auction_round():
    # Initial game information
    game_info = {
        'Round_Number': 1,
        'Total_Number_Items': 3,  # There are 20 items
        'Number_Robots': 5         # 5 robots competing
    }

    # Victory points for the items in this round (random float between 0 and 1)
    item_victory_points = [round(random.uniform(0.0, 1.0), 2) for _ in range(game_info['Total_Number_Items'])]

    # Player information (can be expanded if needed)
    player_information = {}

    # Player's and robots' starting budget (1 unit for each)
    player_budget = 1.0
    robot_budgets = [1.0] * game_info['Number_Robots']

    # Victory points for player and robots
    player_victory_points = 0.0
    robot_victory_points = [0.0] * game_info['Number_Robots']
    # Initialize last auction result to be empty at the start
    last_auction_result = {
        'winner': None,
        'winning_bid': None,
        'vp': None,
        'total': None,
        'min': None
    }
    # Simulating one auction round with 20 items
    for item_idx in range(game_info['Total_Number_Items']):
        # Current item information
        current_info = {
            'Remaining_Number_Items': game_info['Total_Number_Items'] - item_idx,
            'vp': item_victory_points[item_idx],
            'remaining_budget': player_budget
        }

        # Simulate robots' bids as random values between 0.1 and their remaining budget
        robot_bids = [round(random.uniform(0.0, budget), 2) if budget > 0 else 0 for budget in robot_budgets]

        # Player's bid using the sample strategy
        player_bid = auction_bids(game_info, current_info, last_auction_result, player_information)
        player_bid = min(player_bid, player_budget)  # Ensure the bid does not exceed the remaining budget

        # Deduct player's bid no matter if they win or lose
        player_budget -= player_bid
        # Deduct each robot's bid
        for i, bid in enumerate(robot_bids):
            robot_budgets[i] -= bid

        # Determine the winner (highest bid wins)
        all_bids = robot_bids + [player_bid]
        max_bid = max(all_bids)
        min_bid = min(all_bids)

        if max_bid == player_bid:
            winner = 'Player'
            player_victory_points += current_info['vp']  # Add victory points to the player
            print(f"Item {item_idx + 1}: Player wins with bid {player_bid}, Victory Points: {current_info['vp']}")
        else:
            winner_robot_idx = all_bids.index(max_bid)
            winner = f'Robot_{winner_robot_idx + 1}'
            robot_victory_points[winner_robot_idx] += current_info['vp']  # Add victory points to the winning robot
            print(f"Item {item_idx + 1}: {winner} wins with bid {max_bid}, Victory Points: {current_info['vp']}")

        # Update last auction result
        last_auction_result = {
            'winner': winner,
            'winning_bid': max_bid,
            'vp': current_info['vp'],
            'total': sum(all_bids),  # Total of all bids
            'min': min_bid           # Minimum bid submitted
        }

        # Stop the round if the player runs out of budget
        if player_budget <= 0:
            print("Player has run out of budget.")

    # Output the final scores
    print("\nFinal Results:")
    print(f"Player's total victory points: {player_victory_points}")
    for i in range(game_info['Number_Robots']):
        print(f"Robot_{i + 1}'s total victory points: {robot_victory_points[i]}")
    print(f"Player's remaining budget: {player_budget}")
    print(f"Robots' remaining budgets: {robot_budgets}")

# Run the simulation
simulate_auction_round()

{'winner': None, 'winning_bid': None, 'vp': None, 'total': None, 'min': None}
{'total_points_remaining': 6.0, 'Robots': [1.0, 1.0, 1.0, 1.0, 1.0], 'VP': {'average': None, 'std': None}}
Item 1: Robot_3 wins with bid 0.92, Victory Points: 0.96
{'winner': 'Robot_3', 'winning_bid': 0.92, 'vp': 0.96, 'total': 3.0833333333333335, 'min': 0.11}
{'total_points_remaining': 2.9166666666666665, 'Robots': [0.89, 0.42666666666666664, 0.07999999999999996, 0.42666666666666664, 0.42666666666666664], 'VP': {'average': None, 'std': None}, 'last_bid': 0.3333333333333333}
Item 2: Robot_5 wins with bid 0.86, Victory Points: 0.31
{'winner': 'Robot_5', 'winning_bid': 0.86, 'vp': 0.31, 'total': 1.7633333333333334, 'min': 0.06}
{'total_points_remaining': 1.153333333333333, 'Robots': [0.8300000000000001, 0.2566666666666666, -0.09000000000000005, 0.2566666666666666, -0.43333333333333335], 'VP': {'average': None, 'std': None}, 'last_bid': 0.3333333333333333}
Item 3: Player wins with bid 0.3333333333333333, Victory

In [26]:
import numpy as np
import random

def compute_transport_plan(city_info, transport_costs, failure_probs):
    """
    Compute the optimal transportation plan based on city information, transport costs, and failure probabilities.
    
    Parameters:
    - city_info: Nx4 array where each row contains [buying_price, selling_price, max_demand, max_supply]
    - transport_costs: NxN array representing transportation cost between cities
    - failure_probs: NxN array representing failure probabilities for routes between cities
    
    Returns:
    - transport_plan: List of [source_city, destination_city, quantity] instructions
    """
    N = city_info.shape[0]  # Number of cities
    transport_plan = []
    
    # Step 1: Calculate the profit for each route (i to j)
    profit_matrix = np.zeros((N, N))
    for i in range(N):
        for j in range(N):
            if i != j:
                profit_matrix[i, j] = (city_info[j, 1] - city_info[i, 0] - transport_costs[i, j])

    # Step 2: Sort the routes by profit in descending order
    sorted_routes = []
    for i in range(N):
        for j in range(N):
            if i != j:
                sorted_routes.append((i, j, profit_matrix[i, j]))
    
    sorted_routes = sorted(sorted_routes, key=lambda x: x[2], reverse=True)
    
    # Step 3: Transport goods along the most profitable routes
    remaining_supply = city_info[:, 3].copy()  # Max supply for each city
    remaining_demand = city_info[:, 2].copy()  # Max demand for each city
    
    for source, destination, _ in sorted_routes:
        if remaining_supply[source] > 0 and remaining_demand[destination] > 0:
            # Determine the amount to transport (minimum of available supply and demand)
            quantity = min(remaining_supply[source], remaining_demand[destination])
            
            # Add to the transport plan
            transport_plan.append([source, destination, quantity])
            
            # Update supply and demand
            remaining_supply[source] -= quantity
            remaining_demand[destination] -= quantity
    
    return transport_plan



def simulate_transportation(city_info, transport_costs, failure_probs, transport_plan):
    """
    Simulate the transportation based on the given plan and calculate the total profit.

    Parameters:
    - city_info: Nx4 array where each row contains [buying_price, selling_price, max_demand, max_supply]
    - transport_costs: NxN array representing transportation cost between cities
    - failure_probs: NxN array representing failure probabilities for routes between cities
    - transport_plan: List of [source_city, destination_city, quantity] instructions

    Returns:
    - total_profit: The total profit generated from the transportation plan
    - successful_transports: Number of successful transports
    - failed_transports: Number of failed transports
    """

    total_profit = 0
    successful_transports = 0
    failed_transports = 0

    # Iterate over each transport instruction in the plan
    for source, destination, quantity in transport_plan:
        # Get the relevant prices, transport cost, and failure probability
        buy_price = city_info[source][0]
        sell_price = city_info[destination][1]
        transport_cost = transport_costs[source][destination]
        failure_prob = failure_probs[source][destination]

        # Check if the route succeeds (based on failure probability)
        if random.random() > failure_prob:
            # The route succeeds, calculate the profit
            profit = (sell_price - buy_price - transport_cost) * quantity
            total_profit += profit
            successful_transports += 1
            print(f"Transport from city {source} to city {destination} succeeded: Profit = {profit}")
        else:
            # The route fails, no profit or loss is incurred
            failed_transports += 1
            print(f"Transport from city {source} to city {destination} failed.")

    return total_profit, successful_transports, failed_transports

# Example data
city_info = np.array([
    [99, 101, 100, 200],  # City 0: [buy_price, sell_price, max_demand, max_supply]
    [98, 102, 100, 200],  # City 1
    [100, 105, 100, 200], # City 2
    [101, 104, 100, 200]  # City 3
])

transport_costs = np.array([
    [0, 1, 2, 3],  # Cost from city 0 to others
    [1, 0, 4, 5],  # Cost from city 1 to others
    [2, 4, 0, 6],  # Cost from city 2 to others
    [3, 5, 6, 0]   # Cost from city 3 to others
])

failure_probs = np.array([
    [0, 0.1, 0.2, 0.3],  # Failure probabilities from city 0 to others
    [0.1, 0, 0.3, 0.4],  # Failure probabilities from city 1 to others
    [0.2, 0.3, 0, 0.5],  # Failure probabilities from city 2 to others
    [0.3, 0.4, 0.5, 0]   # Failure probabilities from city 3 to others
])

# Example transport plan: [source_city, destination_city, quantity]
transport_plan = [
    [0, 1, 50],  # Transport 50 units from city 0 to city 1
    [1, 2, 30],  # Transport 30 units from city 1 to city 2
    [2, 3, 20],  # Transport 20 units from city 2 to city 3
    [3, 0, 10]   # Transport 10 units from city 3 to city 0
]


transport_plan = compute_transport_plan(city_info, transport_costs, failure_probs)
print("Computed Transport Plan:")
for plan in transport_plan:
    print(f"Transport {plan[2]} units from city {plan[0]} to city {plan[1]}")

# Run the simulation
total_profit, successful_transports, failed_transports = simulate_transportation(city_info, transport_costs, failure_probs, transport_plan)

print("\nFinal Results:")
print(f"Total Profit: {total_profit}")
print(f"Successful Transports: {successful_transports}")
print(f"Failed Transports: {failed_transports}")


Computed Transport Plan:
Transport 100 units from city 0 to city 2
Transport 100 units from city 0 to city 1
Transport 100 units from city 1 to city 0
Transport 100 units from city 1 to city 3
Transport from city 0 to city 2 succeeded: Profit = 400
Transport from city 0 to city 1 succeeded: Profit = 200
Transport from city 1 to city 0 succeeded: Profit = 200
Transport from city 1 to city 3 succeeded: Profit = 100

Final Results:
Total Profit: 900
Successful Transports: 4
Failed Transports: 0
